# Assignment Set 3

*Authors*: Myriam Belkhatir, Salomé Poulain, Shania Sinha

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from matplotlib.animation import FuncAnimation, PillowWriter

## **3.3**: Leapfrog Integration

In [ ]:
import src.leapfrog
reload(src.leapfrog)
from src.leapfrog import Leapfrog

## **I**: Leapfrog Integration for the Simple Harmonic Oscillator

In [ ]:
solver = Leapfrog(T=30)
solver.solve_leapfrog()
solver.plot_results_leapfrog(savefig=False)

In [ ]:
k_values = [0.5, 1.0, 1.5, 2.0, 5.0]

solver_many_k = Leapfrog(T=10)
solver_many_k.solve_leapfrog()

# Plot many positions graphs
solver_many_k.plot_position_many_k(k_values, savefig=False)

# Plot many velocities graphs
solver_many_k.plot_velocity_many_k(k_values, savefig=False)

### ***Bonus***: Leapfrog vs. Runge-Kutta (Energy Conservation Comparison)

In [ ]:
solver_compare = Leapfrog(T=50, dt=0.01)
solver_compare.plot_compare_methods(savefig=False)

## **J**: Sinusoidal Forcing of Harmonic Oscillator

In [ ]:
solver_forcing = Leapfrog(T=30)

omega_value = 0.8

solver_forcing.solve_leapfrog_with_forcing(omega=omega_value)
solver_forcing.plot_results_leapfrog(savefig=False, with_forcing=True, omega=omega_value)

In [ ]:
solver_forcing.plot_phase_space(savefig=False)